In [ ]:
from google.colab import drive
drive.mount('/content/drive')

with open("/content/drive/MyDrive/StoryToGame/writingPrompts/train.wp_source", "r", encoding="utf-8") as f:
    prompts = f.readlines()

with open("/content/drive/MyDrive/StoryToGame/writingPrompts/train.wp_target", "r", encoding="utf-8") as f:
    stories = f.readlines()

print("Prompt:", prompts[0])
print("Story:", stories[0])


Mounted at /content/drive
Prompt: [ WP ] You 've finally managed to discover the secret to immortality . Suddenly , Death appears before you , hands you a business card , and says , `` When you realize living forever sucks , call this number , I 've got a job offer for you . ''

Story: So many times have I walked on ruins , the remainings of places that I loved and got used to.. At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body.. After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly . <newline> It 's funny , but I felt as if after gaining what I desired so long , what I have lived for my entire life , only then , when I achieved immortality I started truly aging . <newline> <newline> 5 world wars have passed , and now 

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "prompt": prompts,
    "story": stories
})

print(df.head())


                                              prompt  \
0  [ WP ] You 've finally managed to discover the...   
1  [ WP ] The moon is actually a giant egg , and ...   
2  [ WP ] You find a rip in time walking through ...   
3  [ WP ] For years in your youth the same imagin...   
4  [ WP ] You glance at your watch 10:34 am , rou...   

                                               story  
0  So many times have I walked on ruins , the rem...  
1  -Week 18 aboard the Depth Reaver , Circa 2023-...  
2  I was feckin ' sloshed , mate . First time I e...  
3  “ No , no no no ... ” She backed up and turned...  
4  There 's a magical moment between wakefulness ...  


In [ ]:
import re

def get_labels(text):
    text = text.lower()
    level_labels, action_labels = [], []


    if re.search(r"forest|woods", text): level_labels.append("forest")
    if re.search(r"castle|fortress|kingdom", text): level_labels.append("castle")
    if re.search(r"enemy|dragon|monster|beast", text): level_labels.append("enemy")
    if re.search(r"treasure|gold|crown|jewel", text): level_labels.append("treasure")
    if re.search(r"lava|fire|volcano", text): level_labels.append("lava")

    if re.search(r"walk|run|travel|journey", text): action_labels.append("walk")
    if re.search(r"jump|leap|climb", text): action_labels.append("jump")
    if re.search(r"attack|fight|battle|sword|strike", text): action_labels.append("attack")

    return ",".join(level_labels), ",".join(action_labels)

df["level_labels"], df["action_labels"] = zip(*df["story"].map(get_labels))
print(df.head(10))


                                              prompt  \
0  [ WP ] You 've finally managed to discover the...   
1  [ WP ] The moon is actually a giant egg , and ...   
2  [ WP ] You find a rip in time walking through ...   
3  [ WP ] For years in your youth the same imagin...   
4  [ WP ] You glance at your watch 10:34 am , rou...   
5                    [ WP ] Through Iron And Flame\n   
6  [ WP ] You live in a world where there has nev...   
7  [ WP ] You are at the park with your kids , wh...   
8  [ WP ] Write a horror story from the perspecti...   
9  [ WP ] An alien invasion happens during an ali...   

                                               story level_labels  \
0  So many times have I walked on ruins , the rem...                
1  -Week 18 aboard the Depth Reaver , Circa 2023-...                
2  I was feckin ' sloshed , mate . First time I e...         lava   
3  “ No , no no no ... ” She backed up and turned...                
4  There 's a magical moment between w

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

X = df["story"]

y = df["level_labels"].apply(lambda x: x.split(",") if x else [])

mlb = MultiLabelBinarizer()
y_bin = mlb.fit_transform(y)

vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X_vec = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y_bin, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

clf = LogisticRegression(max_iter=200)
model = MultiOutputClassifier(clf)
model.fit(X_train, y_train)


MultiOutputClassifier(estimator=LogisticRegression(max_iter=200))

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=mlb.classes_))


              precision    recall  f1-score   support

      castle       0.98      0.57      0.72      1742
       enemy       1.00      0.75      0.86      6128
      forest       1.00      0.64      0.78      2618
        lava       0.80      0.33      0.47      8998
    treasure       1.00      0.62      0.77      4712

   micro avg       0.95      0.55      0.69     24198
   macro avg       0.96      0.58      0.72     24198
weighted avg       0.93      0.55      0.68     24198
 samples avg       0.19      0.17      0.18     24198



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import joblib

joblib.dump(model, "/content/drive/MyDrive/StoryToGame/story_model.joblib")
joblib.dump(vectorizer, "/content/drive/MyDrive/StoryToGame/vectorizer.joblib")
joblib.dump(mlb, "/content/drive/MyDrive/StoryToGame/mlb.joblib")


['/content/drive/MyDrive/StoryToGame/mlb.joblib']